#IMPORTANDO BIBLIOTECAS NECESSÁRIAS


In [41]:
import pandas as pd
import numpy as np

#Carregando Arquivos
##se abrirmos o arquivo previamente em um editor de textos, percerbemos que ele é um arquivo posicional, sem pontos ou vírgulas delimitando as colunas, com isso usaremos o método do pandas, read_fwf()

##A partir do arquivo "LAYOUT DO ARQUIVO – COTAÇÕES HISTÓRICAS" nós temos as informações das delimitações dsa colunas do documento, que são necessárias como parêmetros para o método read_fwf

In [42]:
#Criando lista com as posições das colunas,as quais queremos, a partir do arquivo de referência
#"LAYOUT DO ARQUIVO – COTAÇÕES HISTÓRICAS"
def read_files(path, name_file, year_date, type_file ):

    _file = f'{path}{name_file}{year_date}.{type_file}'

    colspecs = [(2,10), 
                (10,12),
                (12,24),
                (27,39),
                (56,69),
                (69,82),
                (82,95),
                (108,121),
                (152,170),
                (170,188)
    ]
    #Criando lista com os nomes das colunas

    names = ['data_pregao','codbdi','sigla_acao','nome_acao','preco_abertura',
            'preco_maximo','preco_minimo','preco_fechamento','qtd_negocios',
            'volume_negocios']

    df = pd.read_fwf(_file, colspecs = colspecs , 
                    names = names , skiprows = 1)

    return df       


#Transformação dos dados
## conseguindo carregar os dados, vemos algumas alterações que precisam ser feitas, como formatar as datas da coluna 'data_pregao', transformar os valores para float, nas colunas referentes à preço e fazer uma filtragem, pois,para este processo, queremos apenas as ações negocíadas apenas no lote padrão , que  pelo arquivo ""LAYOUT DO ARQUIVO – COTAÇÕES HISTÓRICAS" , sabemos que são aquelas com coddbi = 2


In [43]:
#Filtrando ações pelo lote padrão

def filter_stocks(df):
  df = df[df['codbdi'] == 2]
  df = df.drop(['codbdi'],1)

  return df

In [44]:
#Ajuste do campo de data
def parse_date(df):
  df['data_pregao'] = pd.to_datetime(df['data_pregao'], format = '%Y%m%d')

  return df


In [45]:
#Ajuste dos campos numéricos
def parse_values(df):
  df['preco_abertura']= (df['preco_abertura']/100).astype(float)
  df['preco_maximo']= (df['preco_maximo']/100).astype(float)
  df['preco_minimo']= (df['preco_minimo']/100).astype(float)
  df['preco_fechamento']= (df['preco_fechamento']/100).astype(float)

  return df

In [46]:
# Concatenando os arquivos

def concat_files(path ,name_file ,year_date ,type_file ,final_file ):

  for i, y in enumerate(year_date):
    df = read_files(path, name_file, y, type_file)
    df = filter_stocks(df)
    df = parse_date(df)
    df = parse_values(df)


    if i == 0:
      df_final = df

    else:
      df_final = pd.concat([df_final,df])

  df_final.to_csv(f'{path}//{final_file}', index = False) 


In [47]:
#Executando ETL

year_date = ['2018', '2018', '2020']

path = f'/content/sample_data'

name_file = 'COTAHIST_A'

type_file = 'txt'

final_file = 'all_bovespa.csv'

concat_files(path, name_file,year_date,type_file,final_file)




FileNotFoundError: ignored